In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [12]:
# read the charity_data.csv from the provided cloud URL.
original_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df = original_df.copy()
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [13]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"])

In [14]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [15]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_type_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [16]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < 500].index.tolist()
application_types_to_replace = application_type_counts


In [17]:
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

In [18]:
# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [19]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts = classification_counts[classification_counts < 100].index.tolist()   
classifications_to_replace = classification_counts

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")


In [20]:
# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: count, dtype: int64

In [21]:
# Convert categorical data to numeric with `pd.get_dummies`
# Convert to dummy variables
application_df = pd.get_dummies(application_df, columns=['APPLICATION_TYPE', 'AFFILIATION', 
                                                        'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 
                                                        'SPECIAL_CONSIDERATIONS', 'INCOME_AMT'], drop_first=True)


In [22]:
application_df = application_df.astype(int)
application_df.head()
print(application_df.dtypes)

STATUS                       int32
ASK_AMT                      int32
IS_SUCCESSFUL                int32
APPLICATION_TYPE_T12         int32
APPLICATION_TYPE_T13         int32
APPLICATION_TYPE_T14         int32
APPLICATION_TYPE_T15         int32
APPLICATION_TYPE_T17         int32
APPLICATION_TYPE_T19         int32
APPLICATION_TYPE_T2          int32
APPLICATION_TYPE_T25         int32
APPLICATION_TYPE_T29         int32
APPLICATION_TYPE_T3          int32
APPLICATION_TYPE_T4          int32
APPLICATION_TYPE_T5          int32
APPLICATION_TYPE_T6          int32
APPLICATION_TYPE_T7          int32
APPLICATION_TYPE_T8          int32
APPLICATION_TYPE_T9          int32
AFFILIATION_Family/Parent    int32
AFFILIATION_Independent      int32
AFFILIATION_National         int32
AFFILIATION_Other            int32
AFFILIATION_Regional         int32
CLASSIFICATION_C1200         int32
CLASSIFICATION_C1270         int32
CLASSIFICATION_C1700         int32
CLASSIFICATION_C2000         int32
CLASSIFICATION_C2100

In [23]:
# Split our preprocessed data into our features and target arrays
# Split data
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(columns=['IS_SUCCESSFUL']).values


In [24]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78) # Changed random_state

# Create a StandardScaler instances
# Scale data
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Optimized Model
# Define the model
nn_optimized = Sequential()

# First hidden layer
nn_optimized.add(Input(shape=(X_train_scaled.shape[1],)))

# First hidden layer
nn_optimized.add(Dense(128, activation='relu'))
nn_optimized.add(BatchNormalization())
nn_optimized.add(Dropout(0.3))

# Second hidden layer
nn_optimized.add(Dense(64, activation='relu'))
nn_optimized.add(BatchNormalization())
nn_optimized.add(Dropout(0.3))

# Third hidden layer
nn_optimized.add(Dense(32, activation='relu'))
nn_optimized.add(BatchNormalization())
nn_optimized.add(Dropout(0.3))

# Output layer
nn_optimized.add(Dense(1, activation='sigmoid'))

# Check the structure of the model
nn_optimized.summary()

# Compile the model
optimizer = Adam(learning_rate=0.001) 
nn_optimized.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) # Modified optimizer

# Add early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         6,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,793 (69.50 KB)

 Trainable params: 17,345 (67.75 KB)

 Non-trainable params: 448 (1.75 KB)

In [ ]:
# Train the model
fit_model_optimized = nn_optimized.fit(
    X_train_scaled, y_train,
    epochs=100, 
    batch_size=64, 
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=0
)

In [32]:
# Evaluate the model using the test data
model_loss_optimized, model_accuracy_optimized = nn_optimized.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Optimized Model Loss: {model_loss_optimized}, Optimized Model Accuracy: {model_accuracy_optimized}")

268/268 - 0s - 2ms/step - accuracy: 0.7285 - loss: 0.5488
Optimized Model Loss: 0.548751711845398, Optimized Model Accuracy: 0.7285131216049194
